<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [132]:
import pandas as pd
import psycopg2

In [ ]:
connection = psycopg2.connect(
    dbname='',
    user='',
    host='',
    password='',
    port=
)

cursor = connection.cursor()

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [134]:
# SQL запрос для подсчета вакансий
query = "SELECT COUNT(id) FROM vacancies;"
        
# Выполняем запрос
cursor.execute(query)
        
# Получаем результат
result = cursor.fetchone()
vacancy_count = result[0]
        
print(f"Количество вакансий в базе: {vacancy_count}")


Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [135]:
# SQL запрос для подсчета работодателей
query = "SELECT COUNT(id) FROM employers;"
        
# Выполняем запрос
cursor.execute(query)
        
# Получаем результат
result = cursor.fetchone()
vacancy_count = result[0]
        
print(f"Количество работодателей в базе: {vacancy_count}")

Количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [136]:
# SQL запрос для подсчета регионов
query = "SELECT COUNT(id) FROM areas;"
        
# Выполняем запрос
cursor.execute(query)
        
# Получаем результат
result = cursor.fetchone()
vacancy_count = result[0]
        
print(f"Количество регионов в базе: {vacancy_count}")

Количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [137]:
# SQL запрос для подсчета сфер деятельности
query = "SELECT COUNT(id) FROM industries;"
        
# Выполняем запрос
cursor.execute(query)
        
# Получаем результат
result = cursor.fetchone()
vacancy_count = result[0]
        
print(f"Количество сфер деятельности в базе: {vacancy_count}")

Количество сфер деятельности в базе: 294


***

*Выводы по предварительному анализу данных:*
* В базе данных содержится 49,197 вакансий. Такой большой выбор вакансий говорит об активности на рынке труда и хороших возможностях для соискателей.
* Количество работодателей = 23,501. Подчеркивает разнообразие компаний и организаций на рынке труда.
* Данные покрывают 1,362 региона. Широкий выбор для жителей разных городов и регоинов.
* Сферы деятельности представлены 294 записями. Рабочие места представлены разными направлениями деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [138]:
query = """
        SELECT 
            a.name as area,
            COUNT(v.id) as vacancy_count
        FROM vacancies v
        JOIN areas a ON v.area_id = a.id
        GROUP BY a.name
        ORDER BY vacancy_count DESC
        LIMIT 5;
        """

cursor.execute(query)

results = cursor.fetchall()

print("ТОП-5 РЕГИОНОВ ПО КОЛИЧЕСТВУ ВАКАНСИЙ:")
print("=" * 40)
for region, count in results:
    print(f"{region}: {count} вакансий")
print("=" * 40)

ТОП-5 РЕГИОНОВ ПО КОЛИЧЕСТВУ ВАКАНСИЙ:
Москва: 5333 вакансий
Санкт-Петербург: 2851 вакансий
Минск: 2112 вакансий
Новосибирск: 2006 вакансий
Алматы: 1892 вакансий


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [139]:
query = """
        SELECT
            COUNT(*) "Кол-во вакансий"
        FROM vacancies v
        WHERE
            v.salary_from IS NOT NULL OR
            v.salary_to IS NOT NULL
        """

cursor.execute(query)

results = cursor.fetchall()

salaryborder_count = results[0][0]

print(f"Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: {salaryborder_count}")    

Количество вакансий, где заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [140]:
query = """
SELECT 
    ROUND(AVG(salary_from)) as avg_salary_from,
    ROUND(AVG(salary_to)) as avg_salary_to
FROM vacancies
WHERE 
    (salary_from IS NOT NULL OR salary_to IS NOT NULL)
    AND (salary_from > 0 OR salary_to > 0)
"""

cursor.execute(query)
results = cursor.fetchone()

avg_salary_from = results[0] if results[0] is not None else 0
avg_salary_to = results[1] if results[1] is not None else 0

print("СРЕДНИЕ ЗНАЧЕНИЯ ЗАРПЛАТНОЙ ВИЛКИ:")
print("=" * 40)
print(f"Средняя нижняя граница: {avg_salary_from:,} руб.")
print(f"Средняя верхняя граница: {avg_salary_to:,} руб.")

print("=" * 40)

СРЕДНИЕ ЗНАЧЕНИЯ ЗАРПЛАТНОЙ ВИЛКИ:
Средняя нижняя граница: 71,065 руб.
Средняя верхняя граница: 110,537 руб.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [141]:
query = """
SELECT 
    schedule,
    employment,
    COUNT(*) as vacancy_count
FROM vacancies
WHERE 
    schedule IS NOT NULL AND
    employment IS NOT NULL
GROUP BY schedule, employment
ORDER BY vacancy_count DESC
LIMIT 3;
"""

cursor.execute(query)
results = cursor.fetchall()

print("ТОП-3 САМЫХ ПОПУЛЯРНЫХ СОЧЕТАНИЙ:")
print("=" * 60)
for i, (schedule, employment, count) in enumerate(results, 1):
    print(f"{i}. {schedule:15} / {employment:20} : {count:>6,}")
print("=" * 60)

ТОП-3 САМЫХ ПОПУЛЯРНЫХ СОЧЕТАНИЙ:
1. Полный день     / Полная занятость     : 35,367
2. Удаленная работа / Полная занятость     :  7,802
3. Гибкий график   / Полная занятость     :  1,593


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [142]:
query = """
SELECT 
    experience,
    COUNT(*) as vacancy_count
FROM vacancies
WHERE experience IS NOT NULL
GROUP BY experience
ORDER BY vacancy_count ASC;
"""

cursor.execute(query)
results = cursor.fetchall()

print("ОПЫТ РАБОТЫ: ОТ НАИМЕНЕЕ К НАИБОЛЕЕ ПОПУЛЯРНОМУ:")
print("=" * 50)
for i, (experience, count) in enumerate(results, 1):
    print(f"{i:2}. {experience:25} : {count:>6,} вакансий")
print("=" * 50)

ОПЫТ РАБОТЫ: ОТ НАИМЕНЕЕ К НАИБОЛЕЕ ПОПУЛЯРНОМУ:
 1. Более 6 лет               :  1,337 вакансий
 2. Нет опыта                 :  7,197 вакансий
 3. От 3 до 6 лет             : 14,511 вакансий
 4. От 1 года до 3 лет        : 26,152 вакансий


***

*Выводы по детальному анализу данных:*
* Среди наиболее популярных городов по количеству вакансий довольно города-миллионники и столицы: Москва, Санкт-Петербург, Минск, Новосибирск и Алматы. Это подчеркивает их притягательность для трудоустройства.
* Данные о заработной плате представлены для 24,073 позиций. Достаточно для изучения уровеня оплаты труда и принципов формирования стоимости рабочей силы на рынке.
* Среднее значение зарплатной вилки 71,065 - 110,537. На этот диапазон стоит ориентироваться кандидатам при формировании финансовых ожиданий.
* В топе занятости: Полный день и Полная занятость, затем идут Удаленная работа с Полной занятостью и Гибкий график с Полной занятостью. Все 3 варианта отражают заинтересованность работодателей нанимать специалистов на полную занятость.
* Работодатели отдают предпочтение специалистам с опытом работы от года до трех лет. Можем предположить, что компании стремятся нанимать сотрудников с достаточной квалификацией для решения задач средней сложности.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [143]:
query = """
SELECT 
    e.name as employer_name,
    COUNT(v.id) as vacancy_count
FROM employers e
JOIN vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
ORDER BY vacancy_count DESC
LIMIT 5;
"""

cursor.execute(query)
results = cursor.fetchall()

print("ТОП-5 РАБОТОДАТЕЛЕЙ ПО КОЛИЧЕСТВУ ВАКАНСИЙ")
print("=" * 50)

for i, (employer, count) in enumerate(results, 1):
    print(f"{i}. {employer}: {count:,} вакансий")

print("=" * 50)
if len(results) >= 5:
    first_place = results[0]
    fifth_place = results[4]
    
    print(f"\nПЕРВОЕ МЕСТО: {first_place[0]} - {first_place[1]:,} вакансий")
    print(f"ПЯТОЕ МЕСТО: {fifth_place[0]} - {fifth_place[1]:,} вакансий")
else:
    print("Недостаточно данных для определения 5-го места")
print("=" * 50)

ТОП-5 РАБОТОДАТЕЛЕЙ ПО КОЛИЧЕСТВУ ВАКАНСИЙ
1. Яндекс: 1,933 вакансий
2. Ростелеком: 491 вакансий
3. Тинькофф: 444 вакансий
4. СБЕР: 428 вакансий
5. Газпром нефть: 331 вакансий

ПЕРВОЕ МЕСТО: Яндекс - 1,933 вакансий
ПЯТОЕ МЕСТО: Газпром нефть - 331 вакансий


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [144]:
query = """
        SELECT 
            a.name as region_name,
            COUNT(e.name) employers_count,
            COUNT(v.name) vacancy_count
        FROM areas a
        LEFT JOIN employers e ON a.id = e.area
        LEFT JOIN vacancies v ON a.id = v.area_id
        WHERE v.id IS NULL
        GROUP BY a.id, a.name
        ORDER BY employers_count DESC
        LIMIT 5;
        """
        
cursor.execute(query)
results = cursor.fetchall()

print("ТОП-5 РЕГИОНОВ БЕЗ ВАКАНСИЙ С НАИБОЛЬШИМ КОЛИЧЕСТВОМ РАБОТОДАТЕЛЕЙ")
print("=" * 70)
for i, (region_name, employers_count, vacancy_count) in enumerate(results, 1):
    print(f"{i}. {region_name}: {employers_count} работодателей")
print("=" * 70)

ТОП-5 РЕГИОНОВ БЕЗ ВАКАНСИЙ С НАИБОЛЬШИМ КОЛИЧЕСТВОМ РАБОТОДАТЕЛЕЙ
1. Россия: 410 работодателей
2. Казахстан: 207 работодателей
3. Московская область: 75 работодателей
4. Краснодарский край: 19 работодателей
5. Беларусь: 18 работодателей


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [145]:
query = """
        SELECT 
            e.name as employer_name,
            COUNT(DISTINCT v.area_id) as regions_count
        FROM employers e
        JOIN vacancies v ON e.id = v.employer_id
        WHERE v.area_id IS NOT NULL
        GROUP BY e.id, e.name
        ORDER BY regions_count DESC
        LIMIT 5;
        """
        
cursor.execute(query)
results = cursor.fetchall()

print("РАБОТОДАТЕЛИ ПО КОЛИЧЕСТВУ РЕГИОНОВ")
print("=" * 60)
for i, (employer, regions_count) in enumerate(results, 1):
    print(f"{i:2}. {employer:30} : {regions_count:>2} регионов")
print("=" * 60)

РАБОТОДАТЕЛИ ПО КОЛИЧЕСТВУ РЕГИОНОВ
 1. Яндекс                         : 181 регионов
 2. Ростелеком                     : 152 регионов
 3. Спецремонт                     : 116 регионов
 4. Поляков Денис Иванович         : 88 регионов
 5. ООО ЕФИН                       : 71 регионов


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [146]:
query = """
        SELECT       
            COUNT(*) employers_count
        FROM employers_industries ei
        RIGHT JOIN employers e ON ei.employer_id = e.id
        WHERE ei.industry_id IS NULL
        """

cursor.execute(query)
result = cursor.fetchone()
count_without_industry = result[0]

print("СТАТИСТИКА ПО СФЕРАМ ДЕЯТЕЛЬНОСТИ РАБОТОДАТЕЛЕЙ")
print("=" * 60)
print(f"Работодателей без указанной сферы деятельности: {count_without_industry:,}")
print("=" * 60)

СТАТИСТИКА ПО СФЕРАМ ДЕЯТЕЛЬНОСТИ РАБОТОДАТЕЛЕЙ
Работодателей без указанной сферы деятельности: 8,419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [147]:
query = """
    SELECT
        e.name as company_name
    FROM employers e
    INNER JOIN employers_industries ei ON ei.employer_id = e.id
    GROUP BY e.id, e.name
    HAVING COUNT(ei.industry_id) = 4
    ORDER BY e.name
    OFFSET 2
    LIMIT 1
"""

cursor.execute(query)
result = cursor.fetchone()

if result:
    company_name = result[0]
    
    # Дополнительный запрос для получения отраслей выбранной компании
    industries_query = """
        SELECT i.name
        FROM industries i
        INNER JOIN employers_industries ei ON i.id = ei.industry_id
        INNER JOIN employers e ON e.id = ei.employer_id
        WHERE e.name = %s
        ORDER BY i.name
    """
    
    cursor.execute(industries_query, (company_name,))
    industries = cursor.fetchall()
    
    # Формируем список отраслей
    industry_list = [industry[0] for industry in industries]
    
    print("КОМПАНИЯ НА ТРЕТЬЕМ МЕСТЕ:")
    print("=" * 50)
    print(f"Название: {company_name}")
    print(f"Количество отраслей: {len(industry_list)}")
    print("Отрасли:")
    for industry in industry_list:
        print(f"  • {industry}")
    print("=" * 50)

КОМПАНИЯ НА ТРЕТЬЕМ МЕСТЕ:
Название: 2ГИС
Количество отраслей: 4
Отрасли:
  • Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее)
  • Колл-центры, диспетчерские службы
  • Консалтинговые услуги
  • Производство мультимедиа, контента, редакторская деятельность


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [148]:
query = """
    SELECT
        COUNT(ei.employer_id) employers_count
    FROM employers_industries ei
    LEFT JOIN industries i ON ei.industry_id = i.id
    WHERE i.name = 'Разработка программного обеспечения'
    """

cursor.execute(query)
result = cursor.fetchone()
employers_count = result[0]

print("СТАТИСТИКА ПО СФЕРЕ ДЕЯТЕЛЬНОСТИ")
print("=" * 50)
print(f"Сфера: Разработка программного обеспечения")
print(f"Количество работодателей: {employers_count:,}")
print("=" * 50)

СТАТИСТИКА ПО СФЕРЕ ДЕЯТЕЛЬНОСТИ
Сфера: Разработка программного обеспечения
Количество работодателей: 3,553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [149]:
import requests

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

# Добавляем заголовки для обхода блокировки
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# Скачиваем страницу с заголовками
response = requests.get(url, headers=headers)
response.raise_for_status()

# Читаем таблицы из HTML
table = pd.read_html(response.text)
city_df = table[0]
city_list = tuple(city_df['Город'])

city_list

C:\Users\nsemkina\AppData\Local\Temp\ipykernel_18812\818870879.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(response.text)


('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Краснодар',
 'Самара',
 'Ростов-на-Дону',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [150]:
query = f'''
    (SELECT
        a.name million_region,
        COUNT(v.id) vacancy_count
    FROM vacancies v
    LEFT JOIN employers e on v.employer_id = e.id
    LEFT JOIN areas a on v.area_id = a.id
    WHERE
        e.name = 'Яндекс'
        and a.name in {city_list}
    GROUP BY a.name)

        UNION ALL

    (SELECT 
        'Total', 
        COUNT(*)
    FROM vacancies v
    LEFT JOIN employers e on v.employer_id = e.id
    LEFT JOIN areas a on v.area_id = a.id
    WHERE
        e.name = 'Яндекс'
        and a.name in {city_list})
    
    ORDER BY 2
'''

cursor.execute(query)
results = cursor.fetchall()

# Создаем DataFrame
df = pd.DataFrame(results, columns=['Регион', 'Количество вакансий'])

print("ВАКАНСИИ КОМПАНИИ 'ЯНДЕКС' В ГОРОДАХ-МИЛЛИОНЕРАХ")
print("=" * 50)

# Выводим таблицу без индексов
print(df.to_string())

print("=" * 50)

ВАКАНСИИ КОМПАНИИ 'ЯНДЕКС' В ГОРОДАХ-МИЛЛИОНЕРАХ
             Регион  Количество вакансий
0              Омск                   21
1         Челябинск                   22
2        Красноярск                   23
3         Волгоград                   24
4             Пермь                   25
5            Казань                   25
6    Ростов-на-Дону                   25
7               Уфа                   26
8            Самара                   26
9         Краснодар                   30
10          Воронеж                   32
11      Новосибирск                   35
12  Нижний Новгород                   36
13     Екатеринбург                   39
14  Санкт-Петербург                   42
15           Москва                   54
16            Total                  485


***

*Выводы по анализу работодателей:*
* В топе работодателей по количеству вакансий фигурирует Яндекс, как лидер среди всех с отрывом в почти 4 раза, далее идут примерно ровно Ростелеком, Тинькофф, СБЕР и Газпром нефть (с количеством 300-500 вакансий).
* Для Яндекса также характерно наибольшее покрытие регионов среди других работодателей, среди лидеров далее идут Ростелеком, Спецремонт.
* «Яндекс» разместил 485 вакансий в 16 городах-миллионниках, что подтверждает широкое географическом присутствие компании и возможности трудоустройства в разных регионах.
* Для региона "Россия" наибольшее количество работодателей без активных вакансий. Возможно, такое общее название региона используется работодателями для принадлежности к стране, но не оторажает конкретное место, либо это соответствует возможности удаленного труда в компании.
* Обнаружено 8419 организаций, не указавших свою сферу деятельности. Это говорит о недостаточно полном заполнении информации в базе данных.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [151]:
query = """
    SELECT
        COUNT(v.id) datavacancy_count
    FROM vacancies v
    WHERE
        lower(v.name) LIKE '%данн%' OR
        lower(v.name) LIKE '%data%'
    """

cursor.execute(query)
result = cursor.fetchone()
datavacancy_count = result[0]

print("ОТНОСИТСЯ ЛИ ВАКАНСИЯ К РАБОТЕ С ДАННЫМИ?")
print("=" * 50)
print(f"Содержит в наименовании одно из слов 'data'/'данн'")
print(f"Количество вакансий {datavacancy_count:,}")
print("=" * 50)

ОТНОСИТСЯ ЛИ ВАКАНСИЯ К РАБОТЕ С ДАННЫМИ?
Содержит в наименовании одно из слов 'data'/'данн'
Количество вакансий 1,771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [152]:
query_juniorDS = """
    SELECT
        COUNT(v.id) juniorDS_count
    FROM vacancies v
    WHERE
        (lower(v.name) LIKE '%data scientist%' OR
        lower(v.name) LIKE '%data science%' OR
        lower(v.name) LIKE '%исследователь данных%' OR
        (lower(v.name) LIKE '%ml%' AND lower(v.name) NOT LIKE '%html%') OR
        lower(v.name) LIKE '%machine learning%' OR
        lower(v.name) LIKE '%машинн%обучен%')
            AND
        (lower(v.name) LIKE '%junior%' OR
        lower(v.experience) LIKE '%нет опыта%' OR
        lower(v.employment) LIKE '%стажировка%')
    """
cursor.execute(query_juniorDS)
result = cursor.fetchone()
juniorDS_count = result[0]

print("Количество вакансий для начинающего DS-специалиста")
print("=" * 50)
print(f"Количество вакансий {juniorDS_count:,}")
print("=" * 50)

Количество вакансий для начинающего DS-специалиста
Количество вакансий 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [153]:
query = """
    SELECT
        COUNT(v.id) DSskill_count
    FROM vacancies v
    WHERE
        (lower(v.name) LIKE '%data scientist%' OR
        lower(v.name) LIKE '%data science%' OR
        lower(v.name) LIKE '%исследователь данных%' OR
        (lower(v.name) LIKE '%ml%' AND lower(v.name) NOT LIKE '%html%') OR
        lower(v.name) LIKE '%machine learning%' OR
        lower(v.name) LIKE '%машинн%обучен%')
            AND
        (lower(v.key_skills) LIKE '%sql%' OR
        lower(v.key_skills) LIKE '%postgres%')
    """
cursor.execute(query)
result = cursor.fetchone()
DSskill_count = result[0]

print("Количество вакансий с SQL или postgres в навыках")
print("=" * 50)
print(f"Количество вакансий {DSskill_count:,}")
print("=" * 50)

Количество вакансий с SQL или postgres в навыках
Количество вакансий 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [154]:
query = """
    SELECT
        COUNT(v.id) DSskill_count
    FROM vacancies v
    WHERE
        (lower(v.name) LIKE '%data scientist%' OR
        lower(v.name) LIKE '%data science%' OR
        lower(v.name) LIKE '%исследователь данных%' OR
        (lower(v.name) LIKE '%ml%' AND lower(v.name) NOT LIKE '%html%') OR
        lower(v.name) LIKE '%machine learning%' OR
        lower(v.name) LIKE '%машинн%обучен%')
            AND
        (lower(v.key_skills) LIKE '%python%')
    """
cursor.execute(query)
result = cursor.fetchone()
DSskill_count = result[0]

print("Количество вакансий с Python в навыках")
print("=" * 50)
print(f"Количество вакансий {DSskill_count:,}")
print("=" * 50)

Количество вакансий с Python в навыках
Количество вакансий 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [155]:
query ="""
    SELECT
        ROUND(AVG(
        LENGTH(v.key_skills) - LENGTH(REPLACE(v.key_skills, chr(9), '')) + 1
    ), 2) key_skills_avg
    FROM vacancies v
    WHERE
        (lower(v.name) LIKE '%data scientist%' OR
        lower(v.name) LIKE '%data science%' OR
        lower(v.name) LIKE '%исследователь данных%' OR
        (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
        lower(v.name) LIKE '%machine learning%' OR
        lower(v.name) LIKE '%машинн%обучен%')
        AND v.key_skills IS NOT NULL
        AND v.key_skills != ''
    """
cursor.execute(query)
result = cursor.fetchone()
key_skills_avg = result[0]

print("Ключевые навыки")
print("=" * 50)
print(f"Среднее количество ключевых навыков {key_skills_avg:,}")
print("=" * 50)

Ключевые навыки
Среднее количество ключевых навыков 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [160]:
query ="""
    SELECT
        round(AVG(COALESCE((v.salary_to + v.salary_from)/2, v.salary_to, v.salary_from))) DS_salary_avg
    FROM vacancies v
    WHERE
        (lower(v.name) LIKE '%data scientist%' OR
        lower(v.name) LIKE '%data science%' OR
        lower(v.name) LIKE '%исследователь данных%' OR
        (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') OR
        lower(v.name) LIKE '%machine learning%' OR
        lower(v.name) LIKE '%машинн%обучен%')
        AND (v.salary_to IS NOT NULL OR v.salary_from IS NOT NULL)
        AND v.experience = 'От 3 до 6 лет'
    """
cursor.execute(query)
result = cursor.fetchone()
DS_salary_avg = result[0]

print("Уровень заработной платы")
print("=" * 50)
print(f"Средняя ЗП дата-саентиста {DS_salary_avg:,}")
print("=" * 50)

Уровень заработной платы
Средняя ЗП дата-саентиста 243,115


***


*Выводы по предметному анализу:*
* Результаты показывают, что на рынке есть спрос на вакансии по работе с данными. Число таких вакансий = 1771.
* Для начинающих специалистов в сфере DS доступна 51 подходящая вакансия, есть возможности для старта карьеры в данном направлении.
* В зависимости от ключевых навыков наличие вакансий для специалистов варьируется. Те, у кого указаны SQL/postgre претендуют на 229 вакансию. Для специалистов с Python это значение выше и составляет 357 вакансий. Владение данными языками программирования очень важны в для успешного трудоустройства.
* В серднем для специалиста DS указано чуть более 6 навыков, что может говорить о необходимости владения различными навками для поиска подходящей вакансии на конкурентоспособном уровне.
* Для опытных специалистов DS (опыт 3-6 лет) средняя зарплата составляет 243,115 руб. Такой уровень показывает привлекательность сферы для соискателя.

# Дополнительные исследования

Дополнительно выведем наиболее популярные сферы деятельности по количеству вакансий

In [157]:
import textwrap

query = """
    SELECT
        i.name as industry_name,
        COUNT(DISTINCT e.id) as employers_count,
        COUNT(v.id) as vacancy_count,
        ROUND(COUNT(v.id) * 1.0 / COUNT(DISTINCT e.id), 1) as avg_vacancies_per_employer
    FROM employers_industries ei
    LEFT JOIN industries i ON ei.industry_id = i.id
    LEFT JOIN employers e ON ei.employer_id = e.id
    LEFT JOIN vacancies v ON e.id = v.employer_id
    GROUP BY i.name
    ORDER BY vacancy_count DESC
    LIMIT 5
"""

cursor.execute(query)
results = cursor.fetchall()

print("📊 ТОП-5 СФЕР ДЕЯТЕЛЬНОСТИ")
print("=" * 120)
print(f"{'№':<2} {'Сфера деятельности':<60} {'👔 Работодателей':<12} {'📋 Вакансий':<12} {'📊 Вакансий/работ.'}")
print("-" * 120)

for i, (industry_name, employers_count, vacancy_count, avg_vacancies) in enumerate(results, 1):
    # Переносим длинные названия (максимум 60 символов в строке)
    wrapped_name = textwrap.fill(industry_name, width=60, max_lines=2)
    name_lines = wrapped_name.split('\n')
    
    # Выводим первую строку с данными
    print(f"{i:<2} {name_lines[0]:<60} {employers_count:>10,} {vacancy_count:>12,} {avg_vacancies:>16}")
    
    # Выводим остальные строки названия
    for line in name_lines[1:]:
        print(f"   {line:<35}")

print("=" * 120)

📊 ТОП-5 СФЕР ДЕЯТЕЛЬНОСТИ
№  Сфера деятельности                                           👔 Работодателей 📋 Вакансий   📊 Вакансий/работ.
------------------------------------------------------------------------------------------------------------------------
1  Разработка программного обеспечения                               3,553       12,499              3.5
2  Системная интеграция,  автоматизации технологических и            2,993       11,034              3.7
   бизнес-процессов предприятия, ИТ-консалтинг
3  Интернет-компания (поисковики, платежные системы, соц.сети,       1,675        6,413              3.8
   информационно-познавательные и развлекательные [...]
4  Банк                                                                249        2,742             11.0
5  Мобильная связь                                                     156        1,585             10.2


***

*Выводы:*
* По таблице видно, что наиболее популярной сферой не только по количеству работодателей, но и по количеству вакансий, является "Разработка программного обеспечения". 
* Топ сфер по количеству вакансий можно разделить на более стабильный рынок и меньшее количество вакансий (первые 3 сферы) и на большую текучесть кадров или активынй найм в связи с большим количеством новых вакансий (4 и 5 сферы). 
* Так можно предположить, что сферы с большим количеством вакансий популярны, для первых 3-х сфер не только среди работников, но и среди работодателей. Для Банка и Мобильной связи можно предположить текучесть кадров и/или монополию на рынке среди работодателей.

***

Проведем анализ вакансий для DS специалистов по количеству предложений на рынке в разрезе.

In [ ]:
query = """
    SELECT 
        e.name as company_name,
        COUNT(v.id) as vacancy_count,
        ROUND(AVG(COALESCE((v.salary_to + v.salary_from)/2, v.salary_to, v.salary_from))) as avg_salary,
        MODE() WITHIN GROUP (ORDER BY v.experience) as most_popular_experience,
        MODE() WITHIN GROUP (ORDER BY v.schedule) as most_popular_schedule,
        MODE() WITHIN GROUP (ORDER BY v.employment) as most_popular_employment
    FROM vacancies v
    LEFT JOIN employers e ON v.employer_id = e.id
    WHERE 
        (lower(v.name) LIKE '%data scientist%' OR
         lower(v.name) LIKE '%data science%' OR
         lower(v.name) LIKE '%исследователь данных%' OR
         (lower(v.name) LIKE '%ml%' AND lower(v.name) NOT LIKE '%html%') OR
         lower(v.name) LIKE '%machine learning%' OR
         lower(v.name) LIKE '%машинн%обучен%')
    GROUP BY e.name
    ORDER BY vacancy_count DESC
    LIMIT 10
"""

total_query = """
    SELECT COUNT(*) as total_vacancies
    FROM vacancies v
    WHERE 
        (lower(v.name) LIKE '%data scientist%' OR
         lower(v.name) LIKE '%data science%' OR
         lower(v.name) LIKE '%исследователь данных%' OR
         (lower(v.name) LIKE '%ml%' AND lower(v.name) NOT LIKE '%html%') OR
         lower(v.name) LIKE '%machine learning%' OR
         lower(v.name) LIKE '%машинн%обучен%')
"""

cursor.execute(total_query)
total_vacancies_result = cursor.fetchone()
total_vacancies = total_vacancies_result[0]

# Выполняем основной запрос
cursor.execute(query)
results = cursor.fetchall()

print("🔬 ТОП-10 КОМПАНИЙ ПО ВАКАНСИЯМ DATA SCIENTIST")
print("=" * 150)
print(f"{'№':<3} {'Компания':<25} {'Вакансий':<10} {'Доля %':<8} {'Средняя зарплата':<20} {'Опыт':<20} {'График':<18} {'Тип занятости':<15}")
print("-" * 150)

for i, (company_name, vacancy_count, avg_salary, popular_experience, popular_schedule, popular_employment) in enumerate(results, 1):
    salary_display = f"{avg_salary:,} ₽" if avg_salary else "нет данных"
    
    # Вычисляем долю в процентах
    share_percent = round((vacancy_count / total_vacancies) * 100, 1) if total_vacancies > 0 else 0
    share_display = f"{share_percent}%"
    
    print(f"{i:<3} {company_name:<25} {vacancy_count:^8} {share_display:^8} {salary_display:^20} {popular_experience:<20} {popular_schedule:<18} {popular_employment:^15}")

print("-" * 150)
print(f"{'ВСЕГО':<3} {'':<25} {total_vacancies:^8} {'100%':<8} {'':<20} {'':<20} {'':<18} {'':<15}")
print("=" * 150)

🔬 ТОП-10 КОМПАНИЙ ПО ВАКАНСИЯМ DATA SCIENTIST
№   Компания                  Вакансий   Доля %   Средняя зарплата     Опыт                 График             Тип занятости  
------------------------------------------------------------------------------------------------------------------------------------------------------
1   СБЕР                         39      7.3%        нет данных      От 3 до 6 лет        Полный день        Полная занятость
2   Bell Integrator              25      4.7%        нет данных      Более 6 лет          Удаленная работа   Полная занятость
3   Банк ВТБ (ПАО)               18      3.4%        нет данных      От 1 года до 3 лет   Полный день        Полная занятость
4   VK                           15      2.8%        нет данных      От 1 года до 3 лет   Полный день        Полная занятость
5   Positive Technologies        11      2.1%        нет данных      От 3 до 6 лет        Гибкий график      Полная занятость
6   Яндекс                       9       1.7% 

***

*Выводы*:
* Мы вывели топ-10 компаний по количеству вакансий для специалистов DS.
* Первые 5 компаний - лидеры по доле вакансий для DS, все последующие компании из списка занимают менее 2% от рынка.
* Можно видеть, что практически ни у одной компании для DS специалистов не указана средняя ЗП, это может быть связано с отсутствующими данными в базе, либо с тем, что работодатели не указывают ЗП намеренно, чтобы согласовать ее с наиболее подходящим кадидатом на индивидуальной основе.
* Среди топ-10 выделяется компания Bell Integrator, которая наиболее часто ищет для себя специалистов с опытом более 6 лет. Для остальных компаний ТОПа интересны как спциалисты 1-3 года опыта, так и 3-6 лет.
* Также интересно, что компания Bell Integrator предпочитает нанимать на удаленную работу, в отличие от всех других компаний. 
* Для всех наиболее предпочтителен график на полную занятось.

# Общий вывод по проекту

* В выборке представлено 49197 вакансий от 23501 работодателей. Указана информация по 1362 регионам в рамках 294 сфер деятельности. Наибольшее число предложений сосредоточено в крупных городах, что подчеркивает их ключевую роль на рынке труда и их значимость как экономических центров.
* Зарплаты указаны для 24073 наблюдений. Среди них для нижней границы вилки средним является зарплата в 71,065 руб., для верзней - 110,537 руб.
* Наиболее часто требуемым опытом является стаж от 1 до 3 лет. Преобладающей формой занятости остается полная занятость на полный день.
* Компания "Яндекс" возглавляет список по количеству вакансий, предлагая 1933 рабочих места в 181 регионе.
* В сфере работы с данными найдено 1771 вакансия.
* Более детально для DS-специалистов важны навыки SQL/postgres, но наиболее значимым является Python, он указан как ключевой навык для 357 позиций.
* В среднем DS-специалист должен обладать более чем шестью ключевыми компетенциями, что обеспечивает ему среднюю заработную плату на уровне 243,115 рублей.
* Наибольшее количество вакансий приходится на область разработки программного обеспечения, где зарегистрировано 12499 предложений от 3553 компаний-работодателей.